# Notebook to experiment with the MT5 TSystems' abstractive summarizer model

Author: Hadi, Ver 202304

I am interested in two things: Speed and Quality. Quality itself has three aspects, (i) not fibbing (ii) not cutting off (iii) sufficient info

In [29]:
# LOAD A WEBPAGE FOR ANALYSIS. Extracts pages content as text into `txt` variable

import requests
import html2text  # pip
import lxml.html

#url = "https://www.dw.com/en/brazil-welcomes-russias-lavrov-amid-us-criticism/a-65353778"
#url = "https://www.dw.com/en/apple-opens-first-flagship-store-in-india/a-65354938"

response = requests.get(url)

tree = lxml.html.fromstring(response.text)
el = tree.xpath('//main|//div[@role="main"]|//section[@role="main"]|//div[@class="main-row"]|//div[@class="main"]|//div[@id="main"]')
if el:
    src = "\n".join(el.getall())
    print("found")
else:
    src = response.text
    #print("not found")

h = html2text.HTML2Text()
h.ignore_links = True
h.ignore_emphasis = True
h.images_to_alt = False
h.single_line_break = True
txt = h.handle(src)

# let's remove the fluf manually on this page
if "# Brazil" in txt:
    txt = txt[txt.find("# Brazil welcomes Russia's"):txt.find("mf/wd (AP, AFP)")]
elif "# Apple" in txt:
    txt = txt[txt.find("# Apple opens first"):txt.find("mf/nm (AP, Reuters)")]
print(txt)

# Apple opens first flagship store in India

12 hours ago12 hours ago

Hundreds of people waited in line outside the new Apple store in the financial
hub Mumbai. The company is planning to boost its production in India — the
second-largest smartphone market in the world.

https://p.dw.com/p/4QDnu

Advertisement

Apple CEO Tim Cook was in Mumbai on Tuesday to inaugurate the company's first
retail store in India.

"India has such a beautiful culture and an incredible energy, and we're
excited to build on our long-standing history," Cook said in a statement.

Bloggers, tech analysts and Bollywood celebrities were invited to the opening
of the new store in Mumbai's Jio World Drive shopping mall.

About 300 Apple fans were seen lining up outside the store, some of them
waiting since the previous night.

"The fanboy inside me would not listen," 30-year-old Purav Mehta, who wanted
to get Cook's autograph, told Reuters news agency.

> A second store is set to open in the capital, New Delhi, on

In [18]:
# TODO:  use a method to detect the text language, also fast

print(len(txt), len(txt.split("\n")))  # 3121 chars, 80 lines, xx tokens

1991 52


In [19]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
%time tokenizer = MT5Tokenizer.from_pretrained("T-Systems-onsite/mt5-small-sum-de-en-v2")  # 0.5s, once
%time model = MT5ForConditionalGeneration.from_pretrained("T-Systems-onsite/mt5-small-sum-de-en-v2")  # 4.3s, once
#model.half()  # 92ms, speeds up inference, mem at 708MiB
model.cuda()  # 99ms
# model.eval() unnecessary

!nvidia-smi

CPU times: user 475 ms, sys: 21.2 ms, total: 496 ms
Wall time: 653 ms
CPU times: user 4.16 s, sys: 672 ms, total: 4.83 s
Wall time: 3.62 s
Tue Apr 18 22:18:23 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 4000 with Max-...    Off| 00000000:01:00.0 Off |                  N/A |
| N/A   61C    P0               30W /  N/A|    816MiB /  8192MiB |      0%      Default |
|                  

In [32]:
# mmm, max_new_tokens=20 does good job. without it not so much.

input_ids = tokenizer.encode(txt, return_tensors="pt", max_length=512, truncation=True)  # 3ms

# 625ms on GPU with or without beams
%time output = model.generate(input_ids=input_ids.cuda(), max_new_tokens=120, repetition_penalty=1.5)  # do_sample=True, num_beams=4)

print(tokenizer.decode(output[0], skip_special_tokens=True))

CPU times: user 322 ms, sys: 0 ns, total: 322 ms
Wall time: 322 ms
Russia's foreign minister has expressed gratitude to the South American country for its "excellent" understanding of Ukraine conflict.


In [52]:
txt = """Brazil welcomes Russia's Lavrov amid US criticism

The United States has criticized Brazil's stance on Moscow's invasion of Ukraine and alleged that "Brazil is parroting Russian and Chinese propaganda."

Russian Foreign Minister Sergey Lavrov visited Brazil on Monday and expressed gratitude to the South American country for its controversial efforts to mediate the Ukraine conflict.

"As for the process in Ukraine, we are grateful to our Brazilian friends for their excellent understanding of this situation's genesis. We are grateful (to them) for striving to contribute to finding ways to settle it," Lavrov said, sitting alongside his Brazilian counterpart, Mauro Vieira.

In regards to the Kremlin-backed invasion of Ukraine, Vieira told journalists that Brazil sees sanctions against Russia as causing negative impacts for the global economy. He added that Brazil supports an immediate cease-fire in Ukraine.

Lavrov also met with Brazilian President Luiz Inacio Lula da Silva, who has raised eyebrows in the West with his recent comments on Moscow's invasion of Ukraine.

Lula da Silva: 'Time for China to get involved' in Ukraine peace negotiationsTo view this video please enable JavaScript, and consider upgrading to a web browser that Video Player is loading.Current Time 0:00Duration 0:00Remaining Time 0:00Quality

Brazil cozying up with Russia, China

Brazil has not joined Western nations in imposing sanctions on Russia for its war and has refused requests to supply ammunition to Ukraine.

President Lula da Silva, on a recent trip to the United Arab Emirates, said Kyiv shares the blame for the conflict.

On Saturday, Lula met Chinese leader Xi Jinping in Beijing where he said that "the United States needs to stop encouraging war and start talking about peace. The European Union needs to start talking about peace."

Earlier this month, he also suggested that Ukraine could give up Crimea to end the war.

'If you liberate Crimea, that changes everything'To view this video please enable JavaScript, and consider upgrading to a web browser that Video Player is loading.Current Time 0:00Duration 0:00Remaining Time 0:00Quality

US: Brazil's views are 'deeply problematic'

Brazil's stance on the Ukraine war and its allegations against the West drew sharp criticism from the US, which described the views as "deeply problematic."

Following the meeting between Lavrov and Vieira, US national security spokesman John Kirby said, "Brazil is parroting Russian and Chinese propaganda without at all looking at the facts."

"The most recent comments by Brazil that Ukraine should consider formally ceding Crimea as a peace concession is simply misguided, especially for a country like Brazil that has voted to uphold the principles of sovereignty and territorial integrity (at the UN)," he said.

Brazil, however, rejected the US' comments.

"I don't know how or why he reached that conclusion but I do not agree at all," Brazil's Viera said, referring to the remarks from Kirby.

Lavrov, meanwhile, told a press conference that the West has got itself involved in "a rather tough struggle" to continue its dominance over world affairs.

mf/wd (AP, AFP)

Biden: 'The United States will never, never, never recognize Russia's claims'To view this video please enable JavaScript, and consider upgrading to a web browser that Video Player is loading.Current Time 0:00Duration 0:00Remaining Time 0:00Quality
"""

In [49]:
# TOKENIZER RELATED QUESTIONS/INVESTIGATIONS.

# - use a fast tokenizer to get number of tokens (should be >20tokz, <512tokz) → resolved, below

# import spacy
# SPACY_NLP = spacy.load("de_core_news_sm")
# s = list(SPACY_NLP(txt).sents)[0]
# print(len(s), len(str(s)))
#
# import sentence_transformers
# from transformers import AutoTokenizer
# model = sentence_transformers.SentenceTransformer('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')
# tokz = AutoTokenizer.from_pretrained(('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli'))
# print(tokz.model_max_length)

32 201


512


In [1]:
# TO DO: let's try intel int8 quantization

# TODO PIP etc. copy code from LocalDialog (KICamp code)

from neural_compressor.config import PostTrainingQuantConfig
from optimum.intel.neural_compressor import INCQuantizer, INCModelForSequenceClassification


ModuleNotFoundError: No module named 'neural_compressor'